In [16]:
import os
import pandas as pd

StatementMeta(, 7ab43928-fcc5-4590-840b-5f62ee9f2b55, 18, Finished, Available, Finished)

In [17]:
# Selecionando as tabelas do estágio silver para conversão em arquivos CSV.
tabelas = ["despesas_alucar", "despesas_consigcar", "receita_alucar", "receita_consigcar", "vendas"]
pasta_export = "csv_exports_silver"


# Criar a pasta de exports se não existir
os.makedirs(f"Files/{pasta_export}", exist_ok=True)

for tabela in tabelas:
    try:
        # Ler a tabela do gold (lakehouse atual)
        df = spark.read.table(f"dbo.{tabela}")
        
        # Exportar para pasta temporária (Spark cria nome aleatório)
        temp_path = f"Files/{pasta_export}/{tabela}/"
        df.coalesce(1).write.mode("overwrite").option("header", "true").csv(temp_path)
        
        print(f"✅ {tabela} exportada!")
        
    except Exception as e:
        print(f"❌ Erro com {tabela}: {str(e)}")

print(f"CSVs estão em: Files/{pasta_export}/")
print("Cada tabela terá sua pasta com o arquivo CSV dentro")

StatementMeta(, 7ab43928-fcc5-4590-840b-5f62ee9f2b55, 19, Finished, Available, Finished)

✅ despesas_alucar exportada!
✅ despesas_consigcar exportada!
✅ receita_alucar exportada!
✅ receita_consigcar exportada!
✅ vendas exportada!
CSVs estão em: Files/csv_exports_silver/
Cada tabela terá sua pasta com o arquivo CSV dentro


In [18]:
print("== DESPESAS ALUCAR ==")
df_despesas_alucar = spark.sql("SELECT * FROM gold.dbo.despesas_alucar LIMIT 1000")
display(df_despesas_alucar)

print("== DESPESAS CONSIGCAR ==")
df_despesas_consigcar = spark.sql("SELECT * FROM gold.dbo.despesas_consigcar LIMIT 1000")
display(df_despesas_consigcar)

print("== RECEITA ALUCAR ==")
df_receita_alucar = spark.sql("SELECT * FROM gold.dbo.receita_alucar LIMIT 1000")
display(df_receita_alucar)

print("== RECEITA CONSIGCAR ==")
df_receita_consigcar = spark.sql("SELECT * FROM gold.dbo.receita_consigcar LIMIT 1000")
display(df_receita_consigcar)

print("== VENDAS ==")
df_vendas = spark.sql("SELECT * FROM gold.dbo.vendas LIMIT 1000")
display(df_vendas)

StatementMeta(, 7ab43928-fcc5-4590-840b-5f62ee9f2b55, 20, Finished, Available, Finished)

== DESPESAS ALUCAR ==


SynapseWidget(Synapse.DataFrame, d30de850-0d3d-4b33-9ad4-65fa4c98da62)

== DESPESAS CONSIGCAR ==


SynapseWidget(Synapse.DataFrame, 4830abd0-7c42-4f42-bcb2-ee6384d5f993)

...

SynapseWidget(Synapse.DataFrame, 7fefa791-b828-4abb-a6e7-f38b22eb62f9)

== RECEITA CONSIGCAR ==


SynapseWidget(Synapse.DataFrame, 80bec395-b3f7-4f73-b332-d8554f355ef0)

== VENDAS ==


SynapseWidget(Synapse.DataFrame, f6e73fbf-d046-4854-9e9b-5f48549dcf78)

---

#### Ação 01

**Analisamos a tabela <mark>despesas_alucar</mark> e identificamos que está em formato wide inadequado para Power BI, com meses como colunas e erro de digitação ("FEVEIRO"). Especificamos a transformação necessária para formato long normalizado (22 linhas → 264 linhas), habilitando time intelligence e análises temporais nativas do Power BI.**

In [19]:
# Converter PySpark para Pandas
df = df_despesas_alucar.toPandas()

# Transformar de wide para long
df_long = pd.melt(
   df, 
   id_vars=['NOME'], 
   var_name='Mes_Nome', 
   value_name='Valor_Despesa'
)

# Corrigir erro de digitação
df_long['Mes_Nome'] = df_long['Mes_Nome'].replace('FEVEIRO', 'FEVEREIRO')

# Mapear número do mês
meses_map = {
   'JANEIRO': 1, 'FEVEREIRO': 2, 'MARÇO': 3, 'ABRIL': 4,
   'MAIO': 5, 'JUNHO': 6, 'JULHO': 7, 'AGOSTO': 8,
   'SETEMBRO': 9, 'OUTUBRO': 10, 'NOVEMBRO': 11, 'DEZEMBRO': 12
}
df_long['Mes_Numero'] = df_long['Mes_Nome'].map(meses_map)

# Adicionar colunas extras
df_long['Empresa'] = 'Alucar'
df_long['Ano'] = 2025
df_long['Data_Referencia'] = pd.to_datetime(
   '2025-' + df_long['Mes_Numero'].astype(str).str.zfill(2) + '-01'
)

# Renomear colunas
df_long = df_long.rename(columns={'NOME': 'Categoria_Despesa'})

# Reordenar colunas
cols_ordem = ['Empresa', 'Categoria_Despesa', 'Ano', 'Mes_Numero', 
             'Mes_Nome', 'Valor_Despesa', 'Data_Referencia']
df_final = df_long[cols_ordem]

# Converter pandas de volta para Spark DataFrame
df_spark = spark.createDataFrame(df_final)

# CRIAR TABELA NO FABRIC
df_spark.write.mode("overwrite").saveAsTable("despesas_alucar_normalizada")

# Definir variáveis para o caminho
pasta_export = "csv_exports_gold"
tabela = "despesas_alucar_normalizada"
temp_path = f"Files/{pasta_export}/{tabela}/"

# USAR PYSPARK PARA CRIAR DIRETÓRIO
# Criar diretório usando Spark (isso força a criação da estrutura)
df_spark.coalesce(1).write.mode("overwrite").option("header", "true").csv(temp_path)

print(f"✅ Tabela criada: despesas_alucar_normalizada")
print(f"✅ Pasta criada: {temp_path}")
print(f"✅ Arquivo salvo: {df_final.shape[0]} registros transformados")
print(f"📁 Caminho completo: {temp_path}")

StatementMeta(, 7ab43928-fcc5-4590-840b-5f62ee9f2b55, 21, Finished, Available, Finished)

✅ Tabela criada: despesas_alucar_normalizada
✅ Pasta criada: Files/csv_exports_gold/despesas_alucar_normalizada/
✅ Arquivo salvo: 264 registros transformados
📁 Caminho completo: Files/csv_exports_gold/despesas_alucar_normalizada/


In [20]:
# Visualizando o Dataframe
df_final

StatementMeta(, 7ab43928-fcc5-4590-840b-5f62ee9f2b55, 22, Finished, Available, Finished)

,Empresa,Categoria_Despesa,Ano,Mes_Numero,Mes_Nome,Valor_Despesa,Data_Referencia
0,Alucar,Despesas com consultorias e assessorias,2025,1,JANEIRO,2017.89,2025-01-01
1,Alucar,"Seguros (patrimonial, de saúde, etc.)",2025,1,JANEIRO,349.21,2025-01-01
2,Alucar,Inteligência Artificial,2025,1,JANEIRO,1789.43,2025-01-01
3,Alucar,Impostos e taxas,2025,1,JANEIRO,1956.11,2025-01-01
4,Alucar,Despesas com matéria-prima,2025,1,JANEIRO,628.75,2025-01-01
...,...,...,...,...,...,...,...
259,Alucar,CRM,2025,12,DEZEMBRO,623.19,2025-12-01
260,Alucar,Despesas com softwares e licenças,2025,12,DEZEMBRO,2456.34,2025-12-01
261,Alucar,Mecanicos,2025,12,DEZEMBRO,2789.01,2025-12-01
262,Alucar,Lucas Pereira Costa,2025,12,DEZEMBRO,5823,2025-12-01


---

#### Ação 02

**Analisamos a tabela <mark>despesas_consigcar</mark> e identificamos os mesmos problemas que despesas_alucar**

In [21]:
# Converter PySpark para Pandas
df = df_despesas_consigcar.toPandas()

# Transformar de wide para long
df_long = pd.melt(
   df, 
   id_vars=['NOME'], 
   var_name='Mes_Nome', 
   value_name='Valor_Despesa'
)

# Corrigir erro de digitação
df_long['Mes_Nome'] = df_long['Mes_Nome'].replace('FEVEIRO', 'FEVEREIRO')

# Mapear número do mês
meses_map = {
   'JANEIRO': 1, 'FEVEREIRO': 2, 'MARÇO': 3, 'ABRIL': 4,
   'MAIO': 5, 'JUNHO': 6, 'JULHO': 7, 'AGOSTO': 8,
   'SETEMBRO': 9, 'OUTUBRO': 10, 'NOVEMBRO': 11, 'DEZEMBRO': 12
}
df_long['Mes_Numero'] = df_long['Mes_Nome'].map(meses_map)

# Adicionar colunas extras
df_long['Empresa'] = 'Consigcar'  # ← Diferença principal do Alucar
df_long['Ano'] = 2025
df_long['Data_Referencia'] = pd.to_datetime(
   '2025-' + df_long['Mes_Numero'].astype(str).str.zfill(2) + '-01'
)

# Renomear colunas
df_long = df_long.rename(columns={'NOME': 'Categoria_Despesa'})

# Reordenar colunas
cols_ordem = ['Empresa', 'Categoria_Despesa', 'Ano', 'Mes_Numero', 
             'Mes_Nome', 'Valor_Despesa', 'Data_Referencia']
df_final = df_long[cols_ordem]

# Converter pandas de volta para Spark DataFrame
df_spark = spark.createDataFrame(df_final)

# CRIAR TABELA NO FABRIC
df_spark.write.mode("overwrite").saveAsTable("despesas_consigcar_normalizada")

# Definir variáveis para o caminho
pasta_export = "csv_exports_gold"
tabela = "despesas_consigcar_normalizada"
temp_path = f"Files/{pasta_export}/{tabela}/"

# USAR PYSPARK PARA CRIAR DIRETÓRIO E SALVAR
df_spark.coalesce(1).write.mode("overwrite").option("header", "true").csv(temp_path)

print(f"✅ Tabela criada: despesas_consigcar_normalizada")
print(f"✅ Pasta criada via PySpark: {temp_path}")
print(f"✅ Arquivo salvo: {df_final.shape[0]} registros transformados")
print(f"📁 Caminho completo: {temp_path}")

StatementMeta(, 7ab43928-fcc5-4590-840b-5f62ee9f2b55, 23, Finished, Available, Finished)

✅ Tabela criada: despesas_consigcar_normalizada
✅ Pasta criada via PySpark: Files/csv_exports_gold/despesas_consigcar_normalizada/
✅ Arquivo salvo: 156 registros transformados
📁 Caminho completo: Files/csv_exports_gold/despesas_consigcar_normalizada/


In [22]:
# Visualizando o Dataframe
df_final

StatementMeta(, 7ab43928-fcc5-4590-840b-5f62ee9f2b55, 24, Finished, Available, Finished)

,Empresa,Categoria_Despesa,Ano,Mes_Numero,Mes_Nome,Valor_Despesa,Data_Referencia
0,Consigcar,Fernanda Costa Almeida Lima,2025,1,JANEIRO,6215.43,2025-01-01
1,Consigcar,Marcelo Henrique Oliveira Souza,2025,1,JANEIRO,4823.36,2025-01-01
2,Consigcar,Thiago Eduardo Pereira Lima,2025,1,JANEIRO,1847.35,2025-01-01
3,Consigcar,Trafego,2025,1,JANEIRO,3482.76,2025-01-01
4,Consigcar,Rafael Augusto Souza Lima,2025,1,JANEIRO,2658.9,2025-01-01
...,...,...,...,...,...,...,...
151,Consigcar,Meta Ads,2025,12,DEZEMBRO,1973.5,2025-12-01
152,Consigcar,LinkedIn Ads,2025,12,DEZEMBRO,4890.22,2025-12-01
153,Consigcar,Google Ads,2025,12,DEZEMBRO,6215.43,2025-12-01
154,Consigcar,CRM,2025,12,DEZEMBRO,2658.9,2025-12-01


---

### Ação 03

**Analisamos <mark>receita_alucar</mark> e confirmamos que está em formato adequado para Power BI (88 registros já normalizados).** 

**Identificamos apenas ajustes menores: adicionar coluna EMPRESA e padronizar nomenclatura das colunas.**

In [23]:
# Converter PySpark para Pandas
df = df_receita_alucar.toPandas()

# Adicionar coluna empresa
df['Empresa'] = 'Alucar'

# Renomear colunas para padronização
df = df.rename(columns={
   'NOME': 'Nome_Cliente',
   'DATA': 'Data_Receita',
   'MES': 'Mes_Numero', 
   'ANO': 'Ano',
   'VALOR_RECEITA': 'Valor_Receita'
})

# Reordenar colunas
cols_ordem = ['Empresa', 'Nome_Cliente', 'Data_Receita', 'Ano', 'Mes_Numero', 'Valor_Receita']
df_final = df[cols_ordem]

# Converter de volta para Spark DataFrame
df_spark = spark.createDataFrame(df_final)

# CRIAR TABELA NO FABRIC
df_spark.write.mode("overwrite").saveAsTable("receita_alucar_otimizada")

# Salvar usando PySpark
pasta_export = "csv_exports_gold"
tabela = "receita_alucar_otimizada"
temp_path = f"Files/{pasta_export}/{tabela}/"

# Salvar CSV
df_spark.coalesce(1).write.mode("overwrite").option("header", "true").csv(temp_path)

print(f"✅ Tabela criada: receita_alucar_otimizada")
print(f"✅ Receita Alucar otimizada salva: {df_final.shape[0]} registros")
print(f"📁 Caminho: {temp_path}")

StatementMeta(, 7ab43928-fcc5-4590-840b-5f62ee9f2b55, 25, Finished, Available, Finished)

✅ Tabela criada: receita_alucar_otimizada
✅ Receita Alucar otimizada salva: 88 registros
📁 Caminho: Files/csv_exports_gold/receita_alucar_otimizada/


In [24]:
# Visualizando o Dataframe
df_final

StatementMeta(, 7ab43928-fcc5-4590-840b-5f62ee9f2b55, 26, Finished, Available, Finished)

,Empresa,Nome_Cliente,Data_Receita,Ano,Mes_Numero,Valor_Receita
0,Alucar,Cláudia Almeida Pereira Lima,01/02/2025,2025,2,823.0
1,Alucar,Bruno Henrique Oliveira Silva,01/02/2025,2025,2,2919.0
2,Alucar,Gustavo da Silva Pereira Costa,01/03/2025,2025,3,2921.0
3,Alucar,Cíntia da Silva Costa Pereira,01/04/2025,2025,4,1741.0
4,Alucar,Cristina Martins Oliveira,01/04/2025,2025,4,3919.0
...,...,...,...,...,...,...
83,Alucar,Vanessa Costa Lima Almeida,31/01/2025,2025,1,1987.0
84,Alucar,Daniel Souza Costa Pereira,31/01/2025,2025,1,4673.0
85,Alucar,Sofia Gomes da Silva,31/03/2025,2025,3,1581.0
86,Alucar,Eduardo Lima Souza Costa,31/03/2025,2025,3,4221.0


---

### Ação 04 
**Analisamos <mark>receita_consigcar</mark> e confirmamos formato adequado, identificando apenas necessidade de padronização: adicionar coluna EMPRESA, criar DATA_REFERENCIA e renomear campos.**

In [25]:
# Converter PySpark para Pandas
df = df_receita_consigcar.toPandas()

# 1. Renomear colunas para padronização
df = df.rename(columns={
   'NOME': 'Fonte_Receita',
   'MES': 'Mes_Numero',
   'Valor': 'Valor_Receita'
})

# 2. Adicionar colunas obrigatórias para padronização
df['Empresa'] = 'Consigcar'

# 3. Criar campo DATA_REFERENCIA usando o ano da própria tabela
df['Data_Referencia'] = pd.to_datetime(
   df['Ano'].astype(str) + '-' + df['Mes_Numero'].astype(str).str.zfill(2) + '-01'
)

# 4. Adicionar nome do mês para facilitar filtros
meses_map = {
   1: 'JANEIRO', 2: 'FEVEREIRO', 3: 'MARÇO', 4: 'ABRIL',
   5: 'MAIO', 6: 'JUNHO', 7: 'JULHO', 8: 'AGOSTO',
   9: 'SETEMBRO', 10: 'OUTUBRO', 11: 'NOVEMBRO', 12: 'DEZEMBRO'
}
df['Mes_Nome'] = df['Mes_Numero'].map(meses_map)

# 5. Converter tipos de dados
df['Valor_Receita'] = df['Valor_Receita'].astype(float)

# 6. Adicionar campos calculados para análise
# df['Trimestre'] = ((df['Mes_Numero'] - 1) // 3) + 1
# df['Semestre'] = ((df['Mes_Numero'] - 1) // 6) + 1

# 7. Reordenar colunas
cols_ordem = [
   'Empresa', 'Fonte_Receita', 'Data_Referencia', 'Ano', 
   'Mes_Numero', 'Mes_Nome', 'Valor_Receita'
]
df_final = df[cols_ordem]

# Converter de volta para Spark DataFrame
df_spark = spark.createDataFrame(df_final)

# CRIAR TABELA NO FABRIC
df_spark.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("receita_consigcar_otimizada")

# 8. Salvar usando PySpark
pasta_export = "csv_exports_gold"
tabela = "receita_consigcar_otimizada"
temp_path = f"Files/{pasta_export}/{tabela}/"

# Salvar CSV
df_spark.coalesce(1).write.mode("overwrite").option("header", "true").csv(temp_path)

print(f"✅ Tabela criada: receita_consigcar_otimizada")
print(f"✅ Receita Consigcar otimizada salva: {df_final.shape[0]} registros")
print(f"📁 Caminho: {temp_path}")

StatementMeta(, 7ab43928-fcc5-4590-840b-5f62ee9f2b55, 27, Finished, Available, Finished)

✅ Tabela criada: receita_consigcar_otimizada
✅ Receita Consigcar otimizada salva: 13 registros
📁 Caminho: Files/csv_exports_gold/receita_consigcar_otimizada/


In [26]:
# Visualizando o Dataframe
df_final

StatementMeta(, 7ab43928-fcc5-4590-840b-5f62ee9f2b55, 28, Finished, Available, Finished)

,Empresa,Fonte_Receita,Data_Referencia,Ano,Mes_Numero,Mes_Nome,Valor_Receita
0,Consigcar,Faturamento_ConsigCar,2025-01-01,2025,1,JANEIRO,35918.269531
1,Consigcar,Faturamento_ConsigCar,2025-02-01,2025,2,FEVEREIRO,51472.941406
2,Consigcar,Faturamento_ConsigCar,2025-03-01,2025,3,MARÇO,34994.218750
3,Consigcar,Faturamento_ConsigCar,2025-04-01,2025,4,ABRIL,48182.769531
4,Consigcar,Faturamento_ConsigCar,2025-05-01,2025,5,MAIO,41348.000000
5,Consigcar,Faturamento_ConsigCar,2025-06-01,2025,6,JUNHO,50000.000000
6,Consigcar,Faturamento_ConsigCar,2025-10-01,2025,10,OUTUBRO,86000.000000
7,Consigcar,Faturamento_ConsigCar,2025-11-01,2025,11,NOVEMBRO,95000.000000
8,Consigcar,Faturamento_ConsigCar,2025-12-01,2025,12,DEZEMBRO,104000.000000
9,Consigcar,Faturamento_ConsigCar,2026-01-01,2026,1,JANEIRO,32000.000000


---

### Ação 05

**Analisamos <mark>df_vendas</mark> confirmando formato adequado, mas identificamos erro de tipos de dados na multiplicação.**

**Corrigimos usando `pd.to_numeric()` para garantir valores numéricos antes dos cálculos, renomeamos colunas ('teste'→'Data_Venda'), extraímos campos temporais das strings de data e adicionamos coluna EMPRESA para padronização.**

In [27]:
print(df.columns)

StatementMeta(, 7ab43928-fcc5-4590-840b-5f62ee9f2b55, 29, Finished, Available, Finished)

Index(['Fonte_Receita', 'Ano', 'Mes_Numero', 'Valor_Receita', 'Empresa',
       'Data_Referencia', 'Mes_Nome'],
      dtype='object')


In [28]:
# Converter PySpark para Pandas
df = df_vendas.toPandas()


# Garantir que valores sejam numéricos antes de calcular
df['Valor_Parcela'] = pd.to_numeric(df['Valor_parcela'], errors='coerce').round(2)
df['Qtd_Parcelas'] = pd.to_numeric(df['Quantidade_de_vezes'], errors='coerce')

# Renomear colunas para padronização
df = df.rename(columns={
   'Data_do_Pagamento': 'Data_Venda',
   'Tipo_Produto': 'Tipo_Produto',
   'Nome': 'Nome_Cliente', 
   'WhatsApp': 'WhatsApp',
   'Vendedor': 'Vendedor',
   'Comentarios': 'Comentarios'
})

# Adicionar colunas calculada
df['Empresa'] = 'Geral' # <-- ALTERAR
df['Valor_Total'] = (df['Valor_Parcela'] * df['Qtd_Parcelas']).round(2)

# 5. Adicionar campos temporais (extrair de strings de data existentes)
df['Ano'] = df['Data_Venda'].str.split('/').str[2]
df['Mes'] = df['Data_Venda'].str.split('/').str[1] 
df['Dia'] = df['Data_Venda'].str.split('/').str[0]

# Mapear nome do mês
meses_map = {
   '01': 'JANEIRO', '02': 'FEVEREIRO', '03': 'MARÇO', '04': 'ABRIL',
   '05': 'MAIO', '06': 'JUNHO', '07': 'JULHO', '08': 'AGOSTO', 
   '09': 'SETEMBRO', '10': 'OUTUBRO', '11': 'NOVEMBRO', '12': 'DEZEMBRO'
}
df['Mes_Nome'] = df['Mes'].map(meses_map)

# Tratar valores nulos e limpar dados
df['Comentarios'] = df['Comentarios'].fillna('').astype(str).replace('nan', '')

# Reordenar colunas
cols_ordem = [
   'Empresa', 'Data_Venda', 'Ano', 'Mes', 'Dia', 'Mes_Nome',
   'Nome_Cliente', 'WhatsApp', 'Vendedor',
   'Tipo_Produto', 'Qtd_Parcelas', 'Valor_Parcela', 'Valor_Total',
   'Comentarios'
]
df_final = df[cols_ordem]

# Converter de volta para Spark DataFrame
df_spark = spark.createDataFrame(df_final)

# CRIAR TABELA NO FABRIC
df_spark.write.mode("overwrite").saveAsTable("vendas_otimizada")

# Salvar usando PySpark
pasta_export = "csv_exports_gold"
tabela = "vendas_otimizada"
temp_path = f"Files/{pasta_export}/{tabela}/"

# Salvar CSV
df_spark.coalesce(1).write.mode("overwrite").option("header", "true").csv(temp_path)

print(f"✅ Tabela criada: vendas_otimizada")
print(f"\n✅ Vendas otimizada salva: {df_final.shape[0]} registros")
print(f"📁 Caminho: {temp_path}")

StatementMeta(, 7ab43928-fcc5-4590-840b-5f62ee9f2b55, 30, Finished, Available, Finished)

✅ Tabela criada: vendas_otimizada

✅ Vendas otimizada salva: 207 registros
📁 Caminho: Files/csv_exports_gold/vendas_otimizada/
